In [ ]:
import osmnx as ox
from Facility import *
import networkx as nx
from constants import *

In [ ]:
class Advertisement:
    def __init__(self):
        self.G=ox.graph_from_bbox(north, south, east, west, network_type='drive')
        self.G=nx.convert_node_labels_to_integers(self.G) 
        self.all_pair_short()
        self.add_structure() 
    def all_pair_short(self):
        self.paths=[]
        for i in self.G.nodes:
            self.paths.append(nx.single_source_dijkstra_path_length(self.G,i,weight='length',cutoff=50000))
    def add_structure(self):
        for u,v,n in self.G.edges:
            self.G.edges[u,v,n]['Facilities']=[]
            self.G.edges[u,v,n]['relevant']={}
            self.G.edges[u,v,n]['relevantR']={}
    def add_facility(self,u,v,n,R):
        if not self.G.has_edge(u,v,n):
            return
        self.G.edges[u,v,n]['Facilities'].append(Facility(R))
        self.G.edges[u,v,n]['Facilities'].sort(key=lambda x: x.R, reverse=True)
             
        if len(self.G.edges[u,v,n]['Facilities'])==1:
            self.preprocessR(u,v,n)
        if R<self.G.edges[u,v,n]['Facilities'][0].R:
            return    
        self.preprocess(u,v,n)    
    
    #def get_key(obj):
    #    return obj.R
    def preprocessR(self,u,v,n):
        RTD=0
        Re=50000
        for w in self.paths[v]:
            for z in self.G.adj[w]:
                for n1 in self.G.adj[w][z]:
                    RTD=self.G.edges[w,z,n1]['length']+self.G.edges[u,v,n]['length']+self.paths[v][w]
                    if u in self.paths[z].keys():
                        RTD+=self.paths[z][v]
                        if RTD<=Re and (u,v,n) not in self.G.edges[w,z,n1]['relevantR'].keys():
                            self.G.edges[w,z,n1]['relevantR'].update({(u,v,n):RTD})
                            
                            
    def preprocess(self,u,v,n):
        RTD=0
        Re=self.G.adj[u][v][n]['Facilities'][0].R
        for w in self.paths[v]:
            for z in self.G.adj[w]:
                for n1 in self.G.adj[w][z]:
                    RTD=self.G.edges[w,z,n1]['length']+self.G.edges[u,v,n]['length']+self.paths[v][w]
                    if u in self.paths[z].keys():
                        RTD+=self.paths[z][v]
                        if RTD<=Re and (u,v,n) not in self.G.edges[w,z,n1]['relevant'].keys():
                            self.G.edges[w,z,n1]['relevant'].update({(u,v,n):RTD})
                            del self.G.edges[w,z,n1]['relevantR'][(u,v,n)]
                            
    
    def query(self,u,v,n,l):
        for w,z,n1 in self.G.edges[u,v,n]['relevant'].keys():
            distance= self.G.edges[u,v,n]['relevant'][(w,z,n1)]
            for i in self.G.adj[w][z][n1]['Facilities']:
                Re=i.R
                if distance>Re:
                    break
                print(Re,distance)    
         
            

In [ ]:
Obj=Advertisement() 

In [ ]:
Obj.add_facility(0,3653,0,10000)
Obj.add_facility(0,3653,0,11000)
Obj.add_facility(0,3653,0,12000)
Obj.add_facility(2,115,0,8000)
Obj.add_facility(2,115,0,13000)
Obj.add_facility(0,7,0,2000)

In [ ]:
for u,v,n in Obj.G.edges:
    #if len(Obj.G.edges[u,v,n]['relevantR'])>0:
    print(u,v,n,Obj.G.edges[u,v,n]['relevantR'])
    print(u,v,n,Obj.G.edges[u,v,n]['relevant'])

In [ ]:
print(Obj.G.edges[0,7,0])

In [ ]:
Obj.add_facility(0,3653,0,28000,30)
Obj.add_facility(0,3653,0,36000,30)
Obj.add_facility(0,3653,0,20000,30)
Obj.add_facility(2,115,0,20000,30)
Obj.add_facility(2,115,0,40000,30)


In [ ]:
sorted(dict.items(),key=lambda x:x[1],reverse=True)

In [ ]:
Obj.query(0,7,0,7)

In [ ]:
def user_register(Obj,u,v,n):
    Re=min(Obj.G.edges[u,v,n]['relevant'].values())
    for w in self.paths[v]:
        for z in self.G.adj[w]:
            for n1 in self.G.adj[w][z]:
                RTD=self.G.edges[w,z,n1]['length']+self.G.edges[u,v,n]['length']+self.paths[v][w]
                if u in self.paths[z].keys():
                    RTD+=self.paths[z][v]
                    if RTD<Re and (u,v,n) not in self.G.edges[w,z,n1]['relevant'].keys():
                        self.G.edges[w,z,n1]['relevantU']+=1

In [ ]:
def user_register(Obj,u,v,n,no):
    Obj.G.edges[u,v,n]['Users']+=no
    Re=min(Obj.G.edges[u,v,n]['relevant'].values())
    for w in Obj.paths[v]:
        for z in Obj.G.adj[w]:
            for n1 in Obj.G.adj[w][z]:
                RTD=Obj.G.edges[w,z,n1]['length']+Obj.G.edges[u,v,n]['length']+Obj.paths[v][w]
                if u in Obj.paths[z].keys():
                    RTD+=Obj.paths[z][v]
                    if RTD<Re and (u,v,n): 
                        Obj.G.edges[w,z,n1]['relevantU']+=no
def user_dereister(Obj,u,v,n,no):
    Obj.G.edges[u,v,n]['Users']-=no
    Re=min(Obj.G.edges[u,v,n]['relevant'].values())
    for w in Obj.paths[v]:
        for z in Obj.G.adj[w]:
            for n1 in Obj.G.adj[w][z]:
                RTD=Obj.G.edges[w,z,n1]['length']+Obj.G.edges[u,v,n]['length']+Obj.paths[v][w]
                if u in Obj.paths[z].keys():
                    RTD+=Obj.paths[z][v]
                    if RTD<Re and (u,v,n): 
                        Obj.G.edges[w,z,n1]['relevantU']+=no
                        
def user_query(Obj,u,v,n):
    return Obj.G.edges[u,v,n]['relevantU']

In [ ]:
user_register(Obj,0,7,0)

In [ ]:
user_query(Obj,0,3653,0)